In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
from vectorbtpro import *
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "dark"
vbt.settings.set_theme(theme)
# optional
vbt.settings['plotting']['layout']['width'] = 900
vbt.settings['plotting']['layout']['height'] = 450

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

<style>
.output_wrapper {
    background-color: #282828 !important; /* Replace with your desired color */
    padding: 10px; /* Optional: Add padding for spacing */
}
</style>

In [ ]:
start = "2024-10-01"
end = "2024-10-10"
symbols = ["MES"]
chunk_size = 2

chunks = pth.get_date_chunks(start, end, chunk_size=1)
chunks

[('2024-10-01', '2024-10-02'),
 ('2024-10-02', '2024-10-03'),
 ('2024-10-03', '2024-10-04'),
 ('2024-10-04', '2024-10-05'),
 ('2024-10-05', '2024-10-06'),
 ('2024-10-06', '2024-10-07'),
 ('2024-10-07', '2024-10-08'),
 ('2024-10-08', '2024-10-09'),
 ('2024-10-09', '2024-10-10')]

In [15]:
chunk_start, chunk_end = chunks[0]

df = pth.SierraChartData.pull(symbols, start=chunk_start, end=chunk_end)
df = df.resample("1min")
df.shape

(1440, 1)

In [13]:
# TODO: look into the advice from polakowo and try to simplify it using from_signals.
#       tip -> it's referring to this page -> https://vectorbt.pro/pvt_7a467f6b/cookbook/portfolio/
#       follow that top to bottom and see if you can figure out how to work with stops from within signal_func.
""""
"You can implement anything you want in a from_signals callback,
search for "Breakout strategy by straddling current price with opposing limit orders" example in the cookbook,
it shows how to manage a single stop and target, you can then expand it as you want"
"""

'"\n"You can implement anything you want in a from_signals callback,\nsearch for "Breakout strategy by straddling current price with opposing limit orders" example in the cookbook,\nit shows how to manage a single stop and target, you can then expand it as you want"\n'